# LLM for extracting metadata filters and re-write query

## Import Langchain dependencies

In [143]:
import json
import os
import sys
import boto3
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)
from langchain.retrievers.self_query.opensearch import OpenSearchTranslator
from langchain.llms.bedrock import Bedrock
from langchain_community.chat_models import BedrockChat
from botocore.config import Config
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

## Initialise Anthropic Claude on Bedrock

In [144]:
bedrock_params = {
    "max_tokens_to_sample":2048,
    "temperature":0.0001,
    "top_k":250,
    "top_p":1,
    "stop_sequences":["\\n\\nHuman:"]
}
bedrock_region="us-east-1"
boto3_bedrock = boto3.client(service_name="bedrock-runtime", config=Config(region_name=bedrock_region))

bedrock_anthropic_llm =  BedrockChat(
    client=boto3_bedrock,
    model_id='anthropic.claude-3-sonnet-20240229-v1:0',
    model_kwargs=bedrock_params,
    streaming=False
    )
bedrock_anthropic_llm.model_kwargs = bedrock_params


### Prompt Inputs:
- ### Field names
- ### input query

## Type 1: Zero shot and highlevel filters

In [147]:
prompt1 = """
        You are part of an information system that processes users queries.
        Given a user query you extract information from it that matches a given list of metadata fields.
        The information to be extracted from the query must match the semantics associated with the given metadata fields.
        The information that you extracted from the query will then be used as filters to narrow down the search space
        when querying an index.
        Just include the value of the extracted metadata without including the name of the metadata field.
        The extracted information in 'Extracted metadata' must be returned as a valid JSON structure.
        
        Query: {query}
        
        Metadata fields: {metadata}
        
        Extracted metadata fields: """

In [148]:
query_ = "brown leather shoes for men"
metadata_ = '{"color","category","gender","price","stock","material"}'

region = 'us-east-1'
bedrock_runtime_client = boto3.client('bedrock-runtime',region_name=region)
response = bedrock_runtime_client.invoke_model( 
            modelId= "anthropic.claude-3-sonnet-20240229-v1:0",
            contentType = "application/json",
            accept = "application/json",
   
            body = json.dumps({
                        "anthropic_version": "bedrock-2023-05-31",
                        "max_tokens": 1024,
                        "temperature": 0.001,
                        "top_k": 250,
                        "top_p": 1,
                        "stop_sequences": [
                            "\n\nHuman:"
                        ],
                        "messages": [
                        {
                            "role": "user",
        "content": prompt1.format(metadata = metadata_, query = query_ )                 }
                            ]
                        }
                        
                         )
            )

res = (response.get('body').read()).decode()

print("original query: \n--------------\n"+query_ + "\n")
        
print(json.loads(res)['content'][0]['text'])

original query: 
--------------
brown leather shoes for men

Extracted metadata: {"brown", "shoes", "men", null, null, "leather"}


## Type 2: Few shot and highlevel filters

### Prompt Inputs:
- ### Examples
- ### Field names
- ### input query

In [149]:
prompt2 = """
        You are part of an information system that processes users queries.
        Given a user query you extract information from it that matches a given list of metadata fields.
        The information to be extracted from the query must match the semantics associated with the given metadata fields.
        The information that you extracted from the query will then be used as filters to narrow down the search space
        when querying an index.
        Just include the value of the extracted metadata without including the name of the metadata field.
        The extracted information in 'Extracted metadata' must be returned as a valid JSON structure.
        
        ###
        Example 1:
        Query: "black leather jacket"
        Metadata fields: {{"color", "category"}}
        Extracted metadata fields: {{"color": "black", "category": clothing, "material":leather}}
        ###
        Example 2:
        Query: "trendy shoes for women"
        Metadata fields: {{"gender", "category"}}
        Extracted metadata fields: {{"gender": "female", "category": "footwear"}}
        ###
        Example 3:
        Query: {query}
        Metadata fields: {metadata}
        Extracted metadata fields: """

In [150]:
region = 'us-east-1'
bedrock_runtime_client = boto3.client('bedrock-runtime',region_name=region)
response = bedrock_runtime_client.invoke_model( 
            modelId= "anthropic.claude-3-sonnet-20240229-v1:0",
            contentType = "application/json",
            accept = "application/json",
   
            body = json.dumps({
                        "anthropic_version": "bedrock-2023-05-31",
                        "max_tokens": 1024,
                        "temperature": 0.001,
                        "top_k": 250,
                        "top_p": 1,
                        "stop_sequences": [
                            "\n\nHuman:"
                        ],
                        "messages": [
                        {
                            "role": "user",
        "content": prompt2.format(metadata = '{"color","category","gender","price","stock","material"}',
                                                      query = query_ )                 }
                            ]
                        }
                        
                         )
            )

res = (response.get('body').read()).decode()

print("original query: \n--------------\n"+query_ + "\n")
print(json.loads(res)['content'][0]['text'])

original query: 
--------------
brown leather shoes for men

{
  "color": "brown",
  "category": "footwear",
  "gender": "male",
  "material": "leather"
}


## Type 3: **Few shot and filters with Comparison and logical Operators: Langchain's self query retriever**


### Prompt Inputs:
- ### Examples
- ### Index mapping with field names and description 
- ### Input query

In [151]:
schema = """{{
    "content": "Summary of a retail product",
    "attributes": {{
    "category": {{
        "description": "The category of the product, the available categories are apparel, footwear, outdoors, electronics, beauty, jewelry, accessories, housewares, homedecor, furniture, seasonal, floral, books, groceries, instruments, tools, hot dispensed, cold dispensed, food service and salty snacks",
        "type": "string"
    }},
    "gender_affinity": {{
        "description": "The gender that the product relates to, the choices are Male and Female",
        "type": "string"
    }},
    "material": {{
        "description": "The material type the product is made of",
        "type": "string"
    }},
    "description": {{
        "description": "The detailed description of the product",
        "type": "string"
    }},
    "color": {{
        "description": "The color of the product",
        "type": "string"
    }}
}}
}}"""

In [153]:
examples = [
    {  "i":1,
        "data_source": schema,
        "user_query": "black leather shoes",
        "structured_request": """{{
    "query": "shoes",
    "filter": "and(eq(\"color\", \"black\"), eq(\"material\", \"leather\"), eq(\"category\", \"footwear\"))"
            }}""",
    },
    
        {  "i":2,
        "data_source": schema,
        "user_query": "jackets for men",
        "structured_request": """{{
    "query": "jackets",
    "filter": "and(eq(\"gender_affinity\", \"male\"), eq(\"category\", \"clothing\"))"
            }}""",
    }
]

example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)
example_prompt=PromptTemplate(input_variables=['data_source', 'i', 'structured_request', 'user_query'],
template='<< Example {i}. >>\nData Source:\n{data_source}\n\nUser Query:\n{user_query}\n\nStructured Request:\n{structured_request}\n')
print(example_prompt.format(**examples[0]))

<< Example 1. >>
Data Source:
{{
    "content": "Summary of a retail product",
    "attributes": {{
    "category": {{
        "description": "The category of the product, the available categories are apparel, footwear, outdoors, electronics, beauty, jewelry, accessories, housewares, homedecor, furniture, seasonal, floral, books, groceries, instruments, tools, hot dispensed, cold dispensed, food service and salty snacks",
        "type": "string"
    }},
    "gender_affinity": {{
        "description": "The gender that the product relates to, the choices are Male and Female",
        "type": "string"
    }},
    "material": {{
        "description": "The material type the product is made of",
        "type": "string"
    }},
    "description": {{
        "description": "The detailed description of the product",
        "type": "string"
    }},
    "color": {{
        "description": "The color of the product",
        "type": "string"
    }}
}}
}}

User Query:
black leather shoes

Struc

### Step 2: Add detailed instructions before the few shots

- #### Instructions on Comparison Operators **(eq | ne | gt | gte | lt | lte | contain | like | in | nin)**
- #### Instructions on Logical Operators **(and | or | not)**
- #### Instructions on data types
- #### Instructions on extracting the main subject of the query and filters separately (**Main subject** to be used for **boosting**)
- #### Instructions on **NO Filter**


In [154]:
prefix_ = """
Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not): logical operator
- `statement1`, `statement2`, ... (comparison statements or logical operation statements): one or more statements to apply the operation to

Make sure that you only use the comparators and logical operators listed above and no others.
Make sure that filters only refer to attributes that exist in the data source.
Make sure that filters only use the attributed names with its function names if there are functions applied on them.
Make sure that filters only use format `YYYY-MM-DD` when handling date data typed values.
Make sure that filters take into account the descriptions of attributes and only make comparisons that are feasible given the type of data being stored.
Make sure that filters are only used as needed. If there are no filters that should be applied return "NO_FILTER" for the filter value.
"""

### Step 3: Add the actual query along with schema after prefix prompt instructions

In [155]:
suffix_ = """<< Example 3. >>
Data Source:
{schema}

User Query:
{query}

Structured Request:
"""

### Step 4: Assemble everything to prepare the final prompt

In [156]:
prompt_ = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix=suffix_,
    prefix=prefix_,
    input_variables=["query","schema"],
)

print(prompt_.format(query="Brown leather shoes for men",schema = schema))


Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | no

### Step 5: Generate the raw structured query

In [157]:
region = 'us-east-1'
bedrock_runtime_client = boto3.client('bedrock-runtime',region_name=region)
response = bedrock_runtime_client.invoke_model( 
            modelId= "anthropic.claude-3-sonnet-20240229-v1:0",
            contentType = "application/json",
            accept = "application/json",
   
            body = json.dumps({
                        "anthropic_version": "bedrock-2023-05-31",
                        "max_tokens": 1024,
                        "temperature": 0.001,
                        "top_k": 250,
                        "top_p": 1,
                        "stop_sequences": [
                            "\n\nHuman:"
                        ],
                        "messages": [
                        {
                            "role": "user",
                            "content": prompt_.format(query=query_,schema = schema)                       }
                            ]
                        }
                        
                         )
            )

res = (response.get('body').read()).decode()

inter_query = (json.loads(res)['content'][0]['text'])[7:-3].replace('\\"',"'").replace("\n","")
query_struct = StructuredQueryOutputParser.from_components().parse(inter_query)   

print("original query: \n--------------\n"+query_ + "\n")
print(inter_query)



original query: 
--------------
brown leather shoes for men

{    "query": "shoes",    "filter": "and(eq('color', 'brown'), eq('material', 'leather'), eq('gender_affinity', 'male'), eq('category', 'footwear'))"}


### Step 6: Write custom rules for transforming the raw structured query into OpenSearch query DSL 

In [158]:
opts = OpenSearchTranslator()
for i in opts.allowed_comparators:
   print(i + " -> "+opts._format_func(i))

eq -> term
lt -> lt
lte -> lte
gt -> gt
gte -> gte
contain -> match
like -> fuzzy


In [159]:
query_ = json.loads(json.dumps(opts.visit_structured_query(query_struct)[1]['filter']))
query = json.dumps(opts.visit_structured_query(query_struct)[0])
if('bool' in query_ and 'should' in query_['bool']):
    query_['bool']['should'].append({
                    "match": {
                    
                        "description": query
                    
                    }
                })
else:
    query_['bool']['should'] = {
                    "match": {
                    
                        "description": query
                    
                    }
                }
print(print(json.dumps(query_, indent = 2)))

{
  "bool": {
    "must": [
      {
        "term": {
          "metadata.color.keyword": "brown"
        }
      },
      {
        "term": {
          "metadata.material.keyword": "leather"
        }
      },
      {
        "term": {
          "metadata.gender_affinity.keyword": "male"
        }
      },
      {
        "term": {
          "metadata.category.keyword": "footwear"
        }
      }
    ],
    "should": {
      "match": {
        "description": "\"shoes\""
      }
    }
  }
}
None
